In [22]:
import os
import pycolmap
# import pathlib
import numpy as np



In [23]:
IMAGES_DIR = "frames"
OUTPUT_PATH = "colmap_output"

output_path = pathlib.Path(OUTPUT_PATH)
image_dir = pathlib.Path(IMAGES_DIR)

# [10 %] Run Colmap (https://github.com/colmap/colmap.git) to get the initial point cloud.

In [ ]:
# output_path: pathlib.Path
# image_dir: pathlib.Path

mvs_path = output_path / "mvs"
database_path = output_path / "database.db"

## THIS RUNS COLMAP, SO IS COMMENTED

# pycolmap.extract_features(database_path, image_dir)
# pycolmap.match_exhaustive(database_path)
# maps = pycolmap.incremental_mapping(database_path, image_dir, output_path)
# maps[0].write(output_path)
# # dense reconstruction
# pycolmap.undistort_images(mvs_path, output_path, image_dir)
# pycolmap.patch_match_stereo(mvs_path)  # requires compilation with CUDA
# pycolmap.stereo_fusion(mvs_path / "dense.ply", mvs_path)


## Visualize dense points

In [ ]:
import numpy as np
import viser
from pathlib import Path
from plyfile import PlyData

ply_path = Path("colmap_output/mvs/dense.ply") 
ply_data = PlyData.read(str(ply_path))

vertex = ply_data['vertex']
points = np.column_stack([vertex['x'], vertex['y'], vertex['z']])
colors = np.column_stack([vertex['red'], vertex['green'], vertex['blue']]).astype(np.uint8)

server = viser.ViserServer()

point_cloud = server.scene.add_point_cloud(
    name="/dense_cloud",
    points=points,
    colors=colors,
    point_size=0.05,
)

print(f"Dense reconstruction: {points.shape[0]} points")

point_size_slider = server.gui.add_slider(
    label="Point Size",
    min=0.001,
    max=0.5,
    step=0.001,
    initial_value=0.05,
)

@point_size_slider.on_update
def _(_) -> None:
    point_cloud.point_size = point_size_slider.value

while True:
    pass

╭────── viser (listening *:8090) ───────╮
│             ╷                         │
│   HTTP      │ http://localhost:8090   │
│   Websocket │ ws://localhost:8090     │
│             ╵                         │
╰───────────────────────────────────────╯

Dense reconstruction: 1082040 points


(viser) Connection opened (0, 1 total), 8 persistent messages

(viser) Connection closed (0, 0 total)

KeyboardInterrupt: 

# [60 %] Using the python binding of Colmap (https://github.com/colmap/colmap/tree/main/pycolmap), read the database and split the model into the foreground (your object) and background models.

## Use off-the-shelf segmentation model sam2 and segment out the object of interest.  Note that segmentation masks might not be 100% precise; you would need to aggregate foreground vs background counts estimated from different views for each 3d point.

In [ ]:
...